In [ ]:
# Import all required packages into this workspace

import random
import pandas as pd
import inspect
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
import torch
import gym

from google.colab import files
from pytorch_lightning import LightningModule, Trainer
from tabulate import tabulate


In [ ]:
# TrainingPlan objects hold instructions on how a set of training cycles should be run.
# They can be added to the myRL object and interacted with via the myRL methods.
# See .... for a user manual


class TrainingPlan:

  def __init__(self, algo=None, eps=None, its=None, name=None, env_name=None):

    # Required Imports
    import random
    import pandas as pd
    from google.colab import files
    from pytorch_lightning import LightningModule, Trainer

    # Set Attributes
    self.algo_class = algo
    self.eps = eps
    self.its = its
    self.name = name
    self.env_name = env_name
    self.rewards = []
    self.agents = []

    # Create a random name if not provided
    if name is None:
      self.name = "trainingPlan-" + str(int(random.random()*1e6))

    # Guard against incomplete training plans
    if (algo is None) or (eps is None) or (its is None) or (env_name is None):
      raise Exception("Error: insufficient arguments supplied")
    
    # Guard against wrong argument types
    self.argTypeCheck(algo=algo, eps=eps, its=its, env_name=env_name)

    # Guard against numbers < 0
    if eps < 0 or its < 0:
      raise Exception("eps and its must be > 0")


  # Run the training process
  def run(self, param=None, val=None):

    # Run the algorithm (currently setup for pyTorchLightning only)
    self.rewards=[]
    num_gpus = torch.cuda.device_count()
    for it in range(self.its):

      # If this training plan is being used for tuning by a tuning plan
      if param is not None and val is not None:
        algo = eval("self.algo_class(env_name=self.env_name, " + param + "=" + str(val) + ")")
      else:
        algo = self.algo_class(env_name=self.env_name)
      
      trainer = Trainer(
          gpus=num_gpus,
          max_epochs=self.eps
      )
      trainer.fit(algo)

      # Return the rewards from the training plan
      self.rewards.append(copy.deepcopy(algo.episodic_rewards))
      self.agents.append(copy.deepcopy(algo.agent))
    return self.rewards

  # Argument type check
  def argTypeCheck(self, algo=None, eps=None, its=None, env_name=None):
    import inspect
    if not inspect.isclass(algo):
      raise Exception("Error: argument for 'algo' has wrong type")
    if not isinstance(eps,  int):
      raise Exception("Error: argument for 'eps' has wrong type")
    if not isinstance(its,  int):
      raise Exception("Error: argument for 'its' has wrong type")
    if not isinstance(env_name,  str):
      raise Exception("Error: argument for 'env_name' has wrong type")

  # Give object name when required
  def __str__(self):
    return self.name

In [ ]:
# TuningPlan objects hold instructions on what parameter and arguments to test.
# They can be added to the myRL object and interacted with via the myRL methods.
# See .... for a user manual


class TuningPlan:

  def __init__(self, name=None, param=None, trainingPlan=None, vals=None):

    # Set Attributes
    self.name = name
    self.param = param
    self.trainingPlan = trainingPlan
    self.vals = vals
    self.results = dict()
    self.agents = dict()

    # Guard and warn users of potential errors when using the software
    if (param is None) or (trainingPlan is None) or (vals is None):
      raise Exception("Error: You cannot create a tuning plan without specifying both a parameter to tune and training plan to use, and the vals to test")
    # Guard against wrong types
    if not isinstance(param, str) or not isinstance(trainingPlan, TrainingPlan) or not isinstance(vals, list):
      raise Exception("Error: some arguments provided are of the wrong type")

    # Automatic name generation if no name provided
    if name is None:
      self.name="tuningPlan-" + param

  # Run the tuning tests
  def run(self):
    self.results.clear()
    self.agents.clear()

    for val in self.vals:
      self.trainingPlan.run(param=self.param, val=val)
      self.results[val] = self.trainingPlan.rewards
      self.agents[val] = self.trainingPlan.agents
    return self.results

  # Give object name when required
  def __str(self):
    return self.name

    

In [ ]:
# TestingPlan objects hold instructions on how to test a given agent (actor neural network).
# They can be added to the myRL object and interacted with via the myRL methods.
# See ..... for a user manual


class TestingPlan:

  def __init__(self, name=None, env_name=None, eps=None, agent=None):

    # Set Attributes
    self.name = name
    self.env_name = env_name
    self.env = gym.make(env_name)
    self.eps=eps
    self.agent=agent
    self.rewards = []

    # Automatic name generated if none provided
    if name is None:
      self.name = "testingPlan-" + str(int(random.random()*1e6))

  # Run the test
  def run(self):
    self.rewards = []
    for ep in range(self.eps):
      state = self.env.reset()
      ep_return = 0
      done = False
      while not done:
        action = self.agent(state, epsilon=0)
        next_state, reward, done, info = self.env.step(action)
        exp = (state, action, reward, done, next_state)
        state = next_state
        ep_return += reward
      self.rewards.append(ep_return)
    return self.rewards

  # Give object name when required
  def __str__(self):
    return self.name


In [ ]:
# myRL objects acts as a hub for running RL experiments.
# Developers can either train, manually tune, or test agents in OpenAI environments using this package in google colab.
# A user manual can be found at ...

class myRL:

  def __init__(self):

    # Import required packages
    # from tabulate import tabulate

    # Set attributes
    self.trainingPlans = []
    self.tuningPlans = []
    self.testingPlans = []
    self.trainingPlanResults = dict()
    self.tuningPlanResults = dict()
    self.testingPlanResults = dict()
    self.trainingPlanAgents = dict()
    self.tuningPlanAgents = dict()


#####################################################   TRAINING ########################################################


### PUBLIC 


  # Add a training plan
  def addTrainingPlan(self, trainingPlan=None):

    # Raise error for wrong argument type
    if not isinstance(trainingPlan, TrainingPlan):
      raise Exception("Wrong argument type. Expected: TrainingPlan")
    
    # Check to see if a training plan has already been added with that name
    for trainingPlan_existing in self.trainingPlans:
      if trainingPlan_existing.name == trainingPlan.name:
        raise Exception("A training plan with this name already exists")

    self.trainingPlans.append(trainingPlan)
    self.trainingPlanResults[trainingPlan.name] = trainingPlan.rewards
    return None

  # Remove a training plan
  def removeTrainingPlan(self, name=None):

    # Raise error for wrong argument type
    if not isinstance(name, str):
      raise Exception("Wrong argument type. Expected: str")
    
    # Remove training plan from list and dictionary reference
    for trainingPlan in self.trainingPlans:
      if trainingPlan.name == name:
        self.trainingPlans.remove(trainingPlan)
    del self.trainingPlanResults[name]
    return None

  # Run a training plan
  def runTrainingPlan(self, name=None):

    # Raise error for wrong object argument type
    if not isinstance(name, str):
      raise Exception("Wrong argument type, Expected: String")

    trainingPlan = self.__getTrainingPlan__(name=name)
    trainingPlan.run()
    self.trainingPlanResults[trainingPlan.name] = copy.deepcopy(trainingPlan.rewards)
    self.trainingPlanAgents[trainingPlan.name] = copy.deepcopy(trainingPlan.agents)
    return self.trainingPlanResults[trainingPlan.name]


  # Get the results of a training plan
  def getTrainingPlanResults(self, name=None, it=None, download=False):
    
    # Raise error for wrong argument types
    if not isinstance(name, str):
      raise Exception("Error: wrong argument type for 'name', Expected: String")
    if not isinstance(it, int) and it is not None:
      raise Exception("Error: wrong argument type for 'it', Expected: Int")
    if not isinstance(download, bool) and download is not None:
      raise Exception("Error: wrong argument type 'download', Expected: Bool")

    # Guard against -ve it numbers
    if it is not None:
      if it < 1:
        raise Exception("Error: The iteration argument should be an integer >=1")
      if download:
        csv_file = self.__createExcelFile__(data=self.trainingPlanResults[name], name=name, it=it)
        files.download("/content/" + name + "_it=" + str(it) + ".csv")
      return self.trainingPlanResults[name][it-1]

    # Guard against no training plan
    try:
      self.trainingPlanResults[name]
    except:
      raise Exception("There is no training plan with that name here")
    
    # If 'it' is None, assume all its requested
    if it is None:
      if download:
        csv_file = self.__createExcelFile__(data=self.trainingPlanResults[name], name=name)
        files.download("/content/" + name + ".csv")
      return self.trainingPlanResults[name]
    
    # If 'it' is specified
    if download:
      csv_file = self.__createExcelFile__(data=[self.trainingPlanResults[name][it-1]], name=name)
      files.download("/content/" + name + ".csv")
    return self.trainingPlanResults[trainingPlan.name][it-1]
  

  # Display the different training plans stored by this object
  def showTrainingPlans(self):

    # If no training plans to show
    if self.trainingPlans == []:
      print("There are no training plans held in the myRL object")

    # Display the training plans in a table format
    trainingPlansFormatted = []
    for trainingPlan in self.trainingPlans:
      trainingPlansFormatted.append([])
      trainingPlansFormatted[-1].append(trainingPlan.name)
      trainingPlansFormatted[-1].append(trainingPlan.eps)
      trainingPlansFormatted[-1].append(trainingPlan.its)
    print(tabulate(trainingPlansFormatted, headers=['Name', 'Episodes', 'Iterations']) + "\n\n")


### PRIVATE


  def __getTrainingPlan__(self, name=None):

    # Raise error for wrong object argument type
    if not isinstance(name, str):
      raise Exception("Wrong argument type, Expected: String")

    # Find the training plan
    for i, trainingPlan in enumerate(self.trainingPlans):
      if trainingPlan.name == name:
        return self.trainingPlans[i]
    
    # Guard against no training plan found
    raise Exception("There is no training plan under that name stored in this myRL object")
    return None


  # Get the mean results from a training plan
  def __meanTraining__(self, trainingPlanName):
    mean_data = []
    eps = len(self.trainingPlanResults[trainingPlanName][0])
    its = len(self.trainingPlanResults[trainingPlanName])
    for ep in range(eps):
      sum = 0
      for it in range(its):
        sum += self.trainingPlanResults[trainingPlanName][it][ep]
      mean_data.append(sum / its)
    return mean_data


  # Get the moving average results from a training plan
  def __movingAverageTraining__(self, data, trainingPlanName, moving_average):
    if moving_average == 0:
      return data
    eps = len(self.trainingPlanResults[trainingPlanName][0])
    moving_average_data = []
    for i in range(eps - (moving_average//2)):
      if i > ((moving_average//2)-1):
        lower_lim = i-(moving_average//2)
        upper_lim = i+(moving_average//2)
        moving_average_data.append(np.sum(data[lower_lim:upper_lim]) / moving_average)
    return moving_average_data


#####################################################   TUNING  ########################################################


### PUBLIC


  # Add a tuning plan
  def addTuningPlan(self, tuningPlan=None):

    # Raise error for wrong argument type
    if not isinstance(tuningPlan, TuningPlan):
      raise Exception("Wrong argument type. Expected: TuningPlan")

    # Check to see if a tuning plan has already been added with that name
    for tuningPlan_existing in self.tuningPlans:
      if tuningPlan_existing.name == tuningPlan.name:
        raise Exception("A tuning plan with this name already exists")

    self.tuningPlans.append(tuningPlan)
    self.tuningPlanResults[tuningPlan.name] = copy.deepcopy(tuningPlan.results)
    return None


  # Remove a tuning plan
  def removeTuningPlan(self, name=None):

    # Raise error for wrong argument type
    if not isinstance(name, str):
      raise Exception("Wrong argument type. Expected: str")
    
    # Remove training plan from list and dictionary reference
    for tuningPlan in self.tuningPlans:
      if tuningPlan.name == name:
        self.tuningPlans.remove(tuningPlan)
    del self.tuningPlanResults[name]
    return None


  # Run a tuning plan
  def runTuningPlan(self, name=None):

    # Guard against no training plan
    try:
      self.tuningPlanResults[name]
    except:
      raise Exception("There is no training plan with that name here")
    
    tuningPlan = self.__getTuningPlan__(name=name)
    tuningPlan.run()
    self.tuningPlanResults[name] = copy.deepcopy(tuningPlan.results)
    self.tuningPlanAgents[name] = copy.deepcopy(tuningPlan.agents)
    return self.tuningPlanResults[name]


  # Get the results of the tuning plan
  def getTuningPlanResults(self, name=None, val=None, download=False, it=None):

    # Raise error for wrong argument type
    if not isinstance(name, str) or name is None:
      raise Exception("Wrong argument type for 'name'. Expected: str")
    if (not isinstance(val, int)) and (not isinstance(val, float)) and (val is not None):
      raise Exception("Wrong argument type for 'val'. Expected: int or float")
    if not isinstance(download, bool):
      raise Exception("Wrong argument type for 'download'. Expected: bool")

    # Guard against no tuning plan
    try:
      self.tuningPlanResults[name]
    except:
      raise Exception("There is no tuning plan with that name here")
    
    # If val is None, assume all vals required
    if val is None:
      if download:
        for key in self.tuningPlanResults[name]:
          csv_file = self.__createExcelFile__(data=self.tuningPlanResults[name][key], name=name)
          files.download("/content/" + name + "_val=" + str(key) + ".csv")
      return self.tuningPlanResults[name]

    # # Convert val to a string
    # val = str(val)

    # Val is specified, but its is None, assume all its
    if it is None:
      # Download an return the csv file for the given value of the tuning test results
      if download:
        csv_file = self.__createExcelFile__(data=self.tuningPlanResults[name][val], name=name, val=val)
        files.download("/content/" + name + "_val=" + str(val) + ".csv")
      return self.tuningPlanResults[name][val]
    
    # Val is specified and it is specified
    # Download an return the csv file for the given value of the tuning test results
    if download:
      csv_file = self.__createExcelFile__(data=self.tuningPlanResults[name][val], name=name, it=it, val=val)
      files.download("/content/" + name + "_val=" + str(val) + "_it=" + str(it) +".csv")
    return self.tuningPlanResults[name][val][it-1]
    

  # Display the tuning plans held by this object
  def showTuningPlans(self):

    # If no tuning plans to show
    if self.tuningPlans == []:
      print("There are no training plans held in the myRL object")

    # Display the tuning plans in a table format
    tuningPlansFormatted = []
    for tuningPlan in self.tuningPlans:
      tuningPlansFormatted.append([])
      tuningPlansFormatted[-1].append(tuningPlan.name)
      tuningPlansFormatted[-1].append(tuningPlan.param)
      tuningPlansFormatted[-1].append(tuningPlan.trainingPlan)
      tuningPlansFormatted[-1].append(tuningPlan.vals)
    print(tabulate(tuningPlansFormatted, headers=['Name', 'Parameter', 'Training Plan', 'Value']) + "\n\n")


### PRIVATE


  # Get a tuning plan
  def __getTuningPlan__(self, name=None):

    # Raise error for wrong object argument type
    if not isinstance(name, str):
      raise Exception("Wrong argument type, Expected: String")

    # Find the tuning plan
    for i, tuningPlan in enumerate(self.tuningPlans):
      if tuningPlan.name == name:
        return self.tuningPlans[i]
    
    # Guard against no tuning plan found
    raise Exception("There is no tuning plan under that name stored in this myRL object")
    return None


  # Get the mean results from a single val of a tuning plan
  def __meanTuning__(self, tuningPlanName, val):
    mean_data = []
    eps = len(self.tuningPlanResults[tuningPlanName][val][0])
    its = len(self.tuningPlanResults[tuningPlanName][val])
    for ep in range(eps):
      sum = 0
      for it in range(its):
        sum += self.tuningPlanResults[tuningPlanName][val][it][ep]
      mean_data.append(sum / its)
    return mean_data


  # Get the moving average results of a single val of a tuning plan
  def __movingAverageTuning__(self, data, tuningPlanName, val, moving_average):
    if moving_average == 0:
      return data
    eps = len(self.tuningPlanResults[tuningPlanName][val][0])
    moving_average_data = []
    for i in range(eps - (moving_average//2)):
      if i > ((moving_average//2)-1):
        lower_lim = i-(moving_average//2)
        upper_lim = i+(moving_average//2)
        moving_average_data.append(np.sum(data[lower_lim:upper_lim]) / moving_average)
    return moving_average_data


#####################################################   TESTING  ########################################################


### PUBLIC


  # Add a testing plan
  def addTestingPlan(self, testingPlan=None):

    # Raise error for wrong argument type
    if not isinstance(testingPlan, TestingPlan):
      raise Exception("Wrong argument type. Expected: TestingPlan")

    # Check to see if a testing plan has already been added with that name
    for testingPlan_existing in self.testingPlans:
      if testingPlan_existing.name == testingPlan.name:
        raise Exception("A testing plan with this name already exists")

    self.testingPlans.append(testingPlan)
    self.testingPlanResults[testingPlan.name] = copy.deepcopy(testingPlan.rewards)
    return None

  # Remove a testing plan
  def removeTestingPlan(self, name=None):

    # Raise error for wrong argument type
    if not isinstance(name, str):
      raise Exception("Wrong argument type. Expected: str")
    
    # Remove testing plan from list and dictionary reference
    for testingPlan in self.testingPlans:
      if testingPlan.name == name:
        self.testingPlans.remove(testingPlan)
    del self.testingPlanResults[name]
    return None

  
  # Run the testing plan
  def runTestingPlan(self, name=None):
    testingPlan = self.__getTestingPlan__(name=name)
    testingPlan.run()
    self.testingPlanResults[name] = testingPlan.rewards
    return self.testingPlanResults[name]
  

  # Get testing plan results
  def getTestingPlanResults(self, name=None, download=False):

    # Raise error for wrong argument type
    if not isinstance(name, str) and name is not None:
      raise Exception("Wrong argument type for 'name'. Expected: str")
    if not isinstance(download, bool):
      raise Exception("Wrong argument type for 'download'. Expected: bool")

    # Guard against no testing plan
    try:
      self.testingPlanResults[name]
    except:
      raise Exception("There is no testing plan with that name here")

    # Download an return the csv file for the given value of the tuning test results
    if download:
      csv_file = self.__createExcelFile__(data=[self.testingPlanResults[name]], name=name)
      files.download("/content/" + name + ".csv")

    return self.testingPlanResults[name]


  # Display the testing plans held by this object
  def showTestingPlans(self):

    # If no tuning plans to show
    if self.testingPlans == []:
      print("There are no training plans held in the myRL object")

    # Display the tuning plans in a table format
    testingPlansFormatted = []
    for testingPlan in self.testingPlans:
      testingPlansFormatted.append([])
      testingPlansFormatted[-1].append(testingPlan.name)
      testingPlansFormatted[-1].append(testingPlan.env_name)
      testingPlansFormatted[-1].append(testingPlan.eps)
    print(tabulate(testingPlansFormatted, headers=['Name', 'Environment', 'Number of episodes']) + "\n\n")


### PRIVATE


  # Find and return the testing plan object
  def __getTestingPlan__(self, name=None):

    # Raise error for wrong object argument type
    if not isinstance(name, str):
      raise Exception("Wrong argument type, Expected: String")

    # Find the testing plan
    for i, testingPlan in enumerate(self.testingPlans):
      if testingPlan.name == name:
        return self.testingPlans[i]
    
    # Guard against no testing plan found
    raise Exception("There is no testing plan under that name stored in this myRL object")
    return None


  # Get the mean value of a single test
  def __meanTesting__(self, testingPlanName):
    mean = 0
    eps = len(self.testingPlanResults[testingPlanName])
    sum = 0
    for ep in range(eps):
      sum += self.testingPlanResults[testingPlanName][ep]
    mean = sum / eps
    return mean


#####################################################   AGENT HANDLING  ########################################################


### PUBLIC


  # Find and return an agent
  def getAgent(self, trainingPlanName=None, it=None, tuningPlanName=None, val=None):

    # Raise error for wrong object argument type
    if not isinstance(trainingPlanName, str) and trainingPlanName is not None:
      raise Exception("Wrong argument type for 'trainingPlanName', Expected: String")
    if not isinstance(tuningPlanName, str) and tuningPlanName is not None:
      raise Exception("Wrong argument type for 'tuningPlanName', Expected: String")
    if not isinstance(it, int) and it is not None:
      raise Exception("Wrong argument type for 'it', Expected: int")
    if not (isinstance(val, int) or isinstance(val, float)) and val is not None:
      raise Exception("Wrong argument type for 'val', Expected: int or float")
    if it == None:
      raise Exception("You require an argument for the 'it' parameter")

    # If there is a training plan, assume this is what is required
    if trainingPlanName is not None:

      # Guard against no agents available
      try:
        self.trainingPlanAgents[trainingPlanName]
      except:
        raise Exception("There are have been no agents trained by this training plan")

      #  Select the correct agent to return
      if it == None and len(self.trainingPlanAgents[trainingPlanName]) > 1:
        raise Exception("There is more than one agent linked to this training plan, please specify the iteration used to produced the agent during training")
      elif it == None:
        return copy.deepcopy(self.trainingPlanAgents[trainingPlanName][0])
      else:
        return copy.deepcopy(self.trainingPlanAgents[trainingPlanName][it])
    
    # If there is a training plan, assume this is what is required
    if tuningPlanName is not None:
      if val is None:
        raise Exception("If you want a tuning plan agent you need to specify which parameter value as an argument")

      # Guard against no agents available
      try:
        self.tuningPlanAgents[tuningPlanName]
      except:
        raise Exception("There are have been no agents trained by this tuning plan")

      #  Select the correct agent to return
      if it == None and len(self.tuningPlanAgents[tuningPlanName][val]) > 1:
        raise Exception("There is more than one agent linked to this tuning plan, please specify the iteration used to produced the agent during training")
      elif it == None:
        return self.tuningPlanAgents[tuningPlanName][val][0]
      else:
        return self.tuningPlanAgents[tuningPlanName][val][it]


#####################################################   RESULTS DISPLAY  ########################################################


### PUBLIC


  # Display training plan results
  def displayTrainingResults(self, trainingPlanName=None, it=None, moving_average=0, mean_its=True, figSize=(5, 5), x_label="Episodes", 
                      y_label="Reward", legend_size=15, legend_loc=0, legend=False, titleSize=20, labelSize=15):
    
    # Raise exception against missing parameters
    if trainingPlanName is None:
      raise Exception("Error: Missing 'trainingPlanName' argument")
    
    # Guard against missing results
    if trainingPlanName is not None:
      try:
        self.trainingPlanResults[trainingPlanName]
      except:
        raise Exception("Error: There are no results for a training plan of the name: " + trainingPlanName)
    
    # If the iteration is specified, plot that iteration only
    if it is not None:
      
      # Get the data
      data = self.trainingPlanResults[trainingPlanName][it-1]

      # Create moving average return list
      moving_average_data = self.__movingAverageTraining__(data=data, trainingPlanName=trainingPlanName, moving_average=moving_average)

      # Plot the graph
      plt.figure(figsize=figSize)
      title = trainingPlanName + "_it=" + str(it)
      dataLabel = "Iteration: " + str(it)
      self.__createSubplot__(data=moving_average_data, x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                              subplotNum=1, numSubplots=1, title=title, titleSize=titleSize,
                              labelSize=labelSize, dataLabel=dataLabel, moving_average=moving_average)
      plt.show()
      return None
    
    # If the mean parameter is True, plot the mean of all iterations
    if mean_its:

      # Create mean data list
      mean_data = self.__meanTraining__(trainingPlanName=trainingPlanName)

      # Create moving average data list
      moving_average_data = self.__movingAverageTraining__(data=mean_data, trainingPlanName=trainingPlanName, moving_average=moving_average)

      # Plot the graph
      plt.figure(figsize=figSize)
      title = trainingPlanName + "_mean_of_" + str(len(self.trainingPlanResults[trainingPlanName])) + "_iterations"
      dataLabel = trainingPlanName + "_mean" + "_mean_of_" + str(len(self.trainingPlanResults[trainingPlanName])) + "_iterations"
      self.__createSubplot__(data=moving_average_data, x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                              subplotNum=1, numSubplots=1, title=title, titleSize=titleSize,
                              labelSize=labelSize, dataLabel=dataLabel, moving_average=moving_average)
      plt.show()
      return None
    
    # Else if mean_its is False, plot all its separately
    else:

      # Create the graph
      plt.figure(figsize=figSize)
      title = trainingPlanName + "_all_iterations"
      dataLabel = trainingPlanName + "_all_iterations"

      for data in self.trainingPlanResults[trainingPlanName]:

        # Get random colour of line
        color = self.__getRandomRGBColour__()

        # Create moving average data list
        moving_average_data = self.__movingAverageTraining__(data=data, trainingPlanName=trainingPlanName, moving_average=moving_average)

        # Plot the line
        self.__createSubplot__(data=moving_average_data, x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                                subplotNum=1, numSubplots=len(self.trainingPlanResults[trainingPlanName]), title=title, titleSize=titleSize,
                                labelSize=labelSize, dataLabel=dataLabel, color=color, moving_average=moving_average)
      plt.show()
      return None


  def displayTuningResults(self, tuningPlanName=None, it=None, moving_average=0, mean_its=True, figSize=(5, 5), x_label="Episodes", 
                      y_label="Reward", legend_size=15, legend_loc=0, legend=False, titleSize=20, labelSize=15, val=None):
    
    # Raise exception against missing parameters
    if tuningPlanName is None:
      raise Exception("Error: Missing 'tuningPlanName' argument")
    
    # If 'it' and 'val' provided, then plot specific graph
    if it is not None and val is not None:
      
      # Get data
      data = self.tuningPlanResults[tuningPlanName][val][it-1]

      # Create moving average data list
      moving_average_data = self.__movingAverageTuning__(data=data, tuningPlanName=tuningPlanName, moving_average=moving_average, val=val)

      # Plot the graph
      plt.figure(figsize=figSize)
      title = tuningPlanName + "_val=" + str(val) + "_iteration=" + str(it)
      dataLabel = tuningPlanName
      self.__createSubplot__(data=moving_average_data, x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                              subplotNum=1, numSubplots=1, title=title, titleSize=titleSize,
                              labelSize=labelSize, dataLabel=dataLabel, moving_average=moving_average)
      plt.show()
      return None

    # If no val provided, assume all vals requested
    if val is None and mean_its:

      # Create the graph
      plt.figure(figsize=figSize)
      title = tuningPlanName + "_mean_of_each_vals"

      for key in self.tuningPlanResults[tuningPlanName]:

        # Get random colour of line
        color = self.__getRandomRGBColour__()
        
        # Get data
        mean_data = self.__meanTuning__(tuningPlanName=tuningPlanName, val=key)

        # Create moving average data list
        moving_average_data = self.__movingAverageTuning__(data=mean_data, tuningPlanName=tuningPlanName, moving_average=moving_average, val=key)

        # Set label an plot the line
        dataLabel = "_val=" + str(val) + "_mean_of_" + str(len(self.tuningPlanResults[tuningPlanName])) + "_iterations"
        self.__createSubplot__(data=moving_average_data, x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                                subplotNum=1, numSubplots=1, title=title, titleSize=titleSize,
                                labelSize=labelSize, dataLabel=dataLabel, color=color, moving_average=moving_average)
      plt.show()
      return None
    
    elif val is None and not mean_its:

      # Create the graph
      plt.figure(figsize=figSize)
      title = tuningPlanName + "_all_vals_all_iterations"

      for key in self.tuningPlanResults[tuningPlanName]:
        for i, it in enumerate(self.tuningPlanResults[tuningPlanName][key]):

          # Get random colour of line
          color = self.__getRandomRGBColour__()

          # Get data
          data = self.tuningPlanResults[tuningPlanName][key][i]

          # Create moving average data list
          moving_average_data = self.__movingAverageTuning__(data=data, tuningPlanName=tuningPlanName, moving_average=moving_average, val=key)

          # Set label an plot the line
          dataLabel = "val=" + str(val) + "_it=" + str(i + 1)
          self.__createSubplot__(data=moving_average_data, x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                                  subplotNum=1, numSubplots=1, title=title, titleSize=titleSize,
                                  labelSize=labelSize, dataLabel=dataLabel, color=color, moving_average=moving_average)
      plt.show()
      return None

    # If val provided and mean_its is True
    elif val is not None and mean_its:
      # Create mean data list
      mean_data = self.__meanTuning__(tuningPlanName=tuningPlanName, val=val)

      # Create moving average data list
      moving_average_data = self.__movingAverageTuning__(data=mean_data, tuningPlanName=tuningPlanName, moving_average=moving_average, val=val)

      # Plot the graph
      plt.figure(figsize=figSize)
      title = tuningPlanName  + "_val=" + str(val) + "_mean_of_" + str(len(self.tuningPlanResults[tuningPlanName])) + "_iterations"
      dataLabel = tuningPlanName + "_mean"
      self.__createSubplot__(data=moving_average_data, x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                              subplotNum=1, numSubplots=1, title=title, titleSize=titleSize,
                              labelSize=labelSize, dataLabel=dataLabel, moving_average=moving_average)
      plt.show()
      return None

    # If val is provided and mean_its is False, provide all results for that value
    elif val is not None and not mean_its:

      # Create the graph
      plt.figure(figsize=figSize)
      title = tuningPlanName + "_val=" + str(val) + "_all results"
      data = self.tuningPlanResults[tuningPlanName][val]
      for i, it in enumerate(data):
        
        # Get random colour of line
        color = self.__getRandomRGBColour__()

        # Create moving average data list
        moving_average_data = self.__movingAverageTuning__(data=it, tuningPlanName=tuningPlanName, moving_average=moving_average, val=val)

        # Plot the line
        dataLabel = tuningPlanName + "_val=" + str(val) + "_it=" + str(i+1)
        self.__createSubplot__(data=moving_average_data, x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                                subplotNum=1, numSubplots=1, title=title, titleSize=titleSize,
                                labelSize=labelSize, dataLabel=dataLabel, color=color, moving_average=moving_average)

      plt.show()
      return None


  def displayTestingResults(self, testingPlanName=None, figSize=(5, 5), x_label="Episodes", 
                      y_label="Reward", legend_size=15, legend_loc=0, legend=False, titleSize=20, labelSize=15):
    
    # Raise exception against missing parameters
    if testingPlanName is None:
      raise Exception("Error: Missing 'testingPlanName' argument")

    # calculate mean reward
    mean_reward = self.__meanTesting__(testingPlanName)

    # Plot the graph
    plt.figure(figsize=figSize)
    title = testingPlanName
    dataLabel = testingPlanName
    self.__createSubplot__(data=self.testingPlanResults[testingPlanName], x_label=x_label, y_label=y_label, legend=legend, legend_size=legend_size, legend_loc=legend_loc,
                            subplotNum=1, numSubplots=1, title=title, titleSize=titleSize,
                            labelSize=labelSize, dataLabel=dataLabel)
    plt.show()

    return mean_reward


### PRIVATE


  # Create a subplot to display the results
  def __createSubplot__(self, data=None, x_label="", y_label="", legend=False, legend_size=15, legend_loc=None, subplotNum=1, numSubplots=1, title="", titleSize=20, labelSize=15, dataLabel="", color='blue', moving_average=0):

    x = [i+(moving_average//2) for i in range(len(data))]

    plt.subplot(numSubplots, 1, subplotNum)
    plt.title(title, size=titleSize)
    plt.ylabel(y_label, size=labelSize)
    plt.xlabel(x_label, size=labelSize)
    plt.plot(x, data, color=color, label=dataLabel)
    
    if legend:
      plt.legend(loc=legend_loc, prop={'size' : legend_size})


  # Get a random rgb tuple to use for the graph colour
  def __getRandomRGBColour__(self):
    for i in range(3):
      r = random.uniform(0, 1)
      g = random.uniform(0, 1)
      b = random.uniform(0, 1)
      rgb = (r, g, b)
    return rgb
  

#####################################################   MISC.  ########################################################


### PRIVATE


  # Create an excel file when provided data in the form of a list
  def __createExcelFile__(self, data, name, it=None, val=None):

    # if 'it' is None, assume all iterations wanted
    if it is None:
      num_its = len(data)
      for i in range(num_its):
        df = pd.DataFrame(data[i])
        df.columns = ['It ' +  str(i+1)]
        if i == 0:
          results = df
        else:
          results = pd.concat([results, df], axis=1)
      if val is None:
        return results.to_csv(name + ".csv")
      else:
        return results.to_csv(name + "_val=" + str(val) + ".csv")
    
    # if 'it' is not None, make an excel file with the single iteration
    else:
      df = pd.DataFrame(data[0])
      df.columns = ['It ' +  str(it)]
      if val is None:
        return df.to_csv(name + "_it=" + str(it) + ".csv")
      else:
        return df.to_csv(name + "_val=" + str(val) + "_it=" + str(it) + ".csv")

  